In [1]:
import xarray as xr

file_path = "/Users/diogogomes/Documents/Uni/Hackathon/github/FIRE-HACK/Data/web_rasters/calculated_vars_0.1deg.nc"

ds = xr.open_dataset(file_path)

print(ds)          # Prints dataset structure
print(ds.variables) # Prints variable list

<xarray.Dataset> Size: 800B
Dimensions:    (latitude: 60, longitude: 40)
Coordinates:
  * latitude   (latitude) float64 480B 42.95 42.85 42.75 ... 37.25 37.15 37.05
  * longitude  (longitude) float64 320B -9.95 -9.85 -9.75 ... -6.25 -6.15 -6.05
Data variables:
    *empty*
Frozen({'latitude': <xarray.IndexVariable 'latitude' (latitude: 60)> Size: 480B
array([42.95, 42.85, 42.75, 42.65, 42.55, 42.45, 42.35, 42.25, 42.15, 42.05,
       41.95, 41.85, 41.75, 41.65, 41.55, 41.45, 41.35, 41.25, 41.15, 41.05,
       40.95, 40.85, 40.75, 40.65, 40.55, 40.45, 40.35, 40.25, 40.15, 40.05,
       39.95, 39.85, 39.75, 39.65, 39.55, 39.45, 39.35, 39.25, 39.15, 39.05,
       38.95, 38.85, 38.75, 38.65, 38.55, 38.45, 38.35, 38.25, 38.15, 38.05,
       37.95, 37.85, 37.75, 37.65, 37.55, 37.45, 37.35, 37.25, 37.15, 37.05]), 'longitude': <xarray.IndexVariable 'longitude' (longitude: 40)> Size: 320B
array([-9.95, -9.85, -9.75, -9.65, -9.55, -9.45, -9.35, -9.25, -9.15, -9.05,
       -8.95, -8.85, -8.75, -8.